## This Notebook scrapes review ratings of Gap T-shirts

### Installing and importing required libraries

In [1]:
!pip install selenium

In [2]:
!pip3 install webdriver_manager

In [3]:
# randomising headers
import requests
import random

headers_list = [
    # Firefox 77 Mac
    {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:77.0) Gecko/20100101 Firefox/77.0",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.5",
    "Referer": "https://www.google.com/",
    "DNT": "1",
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1"
    },
    # Chrome 92.0 Win10
    {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.107 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.5",
    "Accept-Encoding": "gzip, deflate, br",
    "Referer": "https://www.google.com/",
    "DNT": "1",
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1"
    },
    # Chrome 91.0 Win10
    {
    "Connection": "keep-alive",
    "DNT": "1",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
    "Sec-Fetch-Site": "none",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Dest": "document",
    "Referer": "https://www.google.com/",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "en-GB,en-US;q=0.9,en;q=0.8"
    },
    # Firefox 90.0 Win10
    {
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:90.0) Gecko/20100101 Firefox/90.0",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
    "Sec-Fetch-Site": "same-origin",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-User": "?1",
    "Sec-Fetch-Dest": "document",
    "Referer": "https://www.google.com/",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "en-US,en;q=0.9"
    }
]

In [4]:
# getting urls for the sites to scrape
urls = []
mens_shirt = "https://www.gap.com/browse/category.do?cid=5225#pageId=0&style="
styles = [5228,1142578,1053999]
for i in styles:
    urls.append(mens_shirt + str(i) + "&department=75")
urls

['https://www.gap.com/browse/category.do?cid=5225#pageId=0&style=5228&department=75',
 'https://www.gap.com/browse/category.do?cid=5225#pageId=0&style=1142578&department=75',
 'https://www.gap.com/browse/category.do?cid=5225#pageId=0&style=1053999&department=75']

In [ ]:
### extracting links for products from pages

In [5]:
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.service import Service
from webdriver_manager.firefox import GeckoDriverManager
import time
import random

firefox_options = Options()
#firefox_options.add_argument('--headless')
firefox_options.add_argument('--no-sandbox')
firefox_options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Firefox(service=Service(GeckoDriverManager().install()), options = firefox_options)

time.sleep(3)
driver.quit()

# Only one browser will open and stay open until the code is done running.
driver = webdriver.Firefox(service=Service(GeckoDriverManager().install()), options = firefox_options)

product_links = []
product_names = []

for url in urls[0:1]:
    
    driver.get(url) # open 'url' in the already opened browser
    
    assert 'Gap' in driver.title # Wait for the page to load
    
    SCROLL_PAUSE_TIME = 30

    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    wait = WebDriverWait(driver, 30)
    wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, '.site-footer_sublinks--span')))
  

    html = driver.page_source # Get the html of the page

    soup = bs(html, 'html.parser')
    
    for html_class in soup.find_all('div', {'data-testid': 'grid-root'}):
        l = html_class.find('a')
        if l is not None:
            link = l.get('href')
        if link is not None:
            product_links.append(link)
        
        product_name = html_class.find("div","category-page-ozrboz")
        if product_name:
            product_names.append(product_name.text)
        else:
            product_names.append("N/A")  
     
   

driver.quit() # Closing the browser.



# Print all the product links
print('No. of links - ', len(product_links))
for link in product_links:
    print(link)

C:\Users\akshi\AppData\Local\Temp\ipykernel_22688\2556195810.py:17: DeprecationWarning: Firefox will soon stop logging to geckodriver.log by default; Specify desired logs with log_output
  driver = webdriver.Firefox(service=Service(GeckoDriverManager().install()), options = firefox_options)
C:\Users\akshi\AppData\Local\Temp\ipykernel_22688\2556195810.py:23: DeprecationWarning: Firefox will soon stop logging to geckodriver.log by default; Specify desired logs with log_output
  driver = webdriver.Firefox(service=Service(GeckoDriverManager().install()), options = firefox_options)


No. of links -  42
https://www.gap.com/browse/product.do?pid=428038052&cid=5225&pcid=5225&vid=1&cpos=0&cexp=2859&kcid=CategoryIDs%3D5225&cvar=25416&ctype=Listing&cpid=res23100314544580731794508&ccam=25791
https://www.gap.com/browse/product.do?pid=428038052&cid=5225&pcid=5225&vid=1&cpos=0&cexp=2859&kcid=CategoryIDs%3D5225&cvar=25416&ctype=Listing&cpid=res23100314544580731794508&ccam=25791
https://www.gap.com/browse/product.do?pid=774839032&cid=5225&pcid=5225&vid=1&cpos=1&cexp=2859&kcid=CategoryIDs%3D5225&cvar=25416&ctype=Listing&cpid=res23100314544580731794508&ccam=25791
https://www.gap.com/browse/product.do?pid=774842012&cid=5225&pcid=5225&vid=1&cpos=2&cexp=2859&kcid=CategoryIDs%3D5225&cvar=25416&ctype=Listing&cpid=res23100314544580731794508&ccam=25791
https://www.gap.com/browse/product.do?pid=774842042&cid=5225&pcid=5225&vid=1&cpos=4&cexp=2859&kcid=CategoryIDs%3D5225&cvar=25416&ctype=Listing&cpid=res23100314544580731794508&ccam=25791
https://www.gap.com/browse/product.do?pid=774842032

In [9]:
## extracting variables from GAP links
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.service import Service
from webdriver_manager.firefox import GeckoDriverManager
import time
import random


firefox_options = Options()
#firefox_options.add_argument('--headless')
firefox_options.add_argument('--no-sandbox')
firefox_options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Firefox(service=Service(GeckoDriverManager().install()), options = firefox_options)

time.sleep(3)
driver.quit()

# Only one browser will open and stay open until the code is done running.
driver = webdriver.Firefox(service=Service(GeckoDriverManager().install()), options = firefox_options)

fiveStars = []
fourStars = []
threeStars = []
twoStars = []
oneStar = []
Count = []
Price = []

for url in product_links[0:23]:
    
    
    driver.get(url) # open 'url' in the already opened browser
    
    assert 'Gap' in driver.title # Wait for the page to load
    
    SCROLL_PAUSE_TIME = 10

    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    wait = WebDriverWait(driver, 10)
    wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR,'.site-footer_sublinks--span' )))
  

    html = driver.page_source # Get the html of the page

    soup = bs(html, 'html.parser')
    rating = soup.find("span",class_ = "pr-snippet-rating-count") 
   
    
    # getting the price of product
    price= soup.find("div", class_ = "pdp-pricing pdp-mfe-1x0pbuu")
    Rate=price.text
    if Rate is not None :
        Price.append(Rate)
    else :
        Price.append("NA")
    if rating is not None:
    # Extract the text content and append it to the 'Count' list
       Count.append(rating.text)
    else:
        Count.append("N/A")
    # extract rating stars from reviews
    while soup.find('li',class_="pr-ratings-histogram-list-item") is not None:
        Stars = []
        stars = soup.find('li',"pr-ratings-histogram-list-item")
        if stars is not None:
            Stars.append(stars.text)
        else :
            Stars.append("N/A")
        stars.extract()
        fiveStars.append(Stars[0])
        fourStars.append(Stars[1])
        threeStars.append(Stars[2])
        twoStars.append(Stars[3])
        oneStar.append(Stars[4])    
    
driver.quit() # Closing the browser    



C:\Users\akshi\AppData\Local\Temp\ipykernel_22688\1526353581.py:18: DeprecationWarning: Firefox will soon stop logging to geckodriver.log by default; Specify desired logs with log_output
  driver = webdriver.Firefox(service=Service(GeckoDriverManager().install()), options = firefox_options)
C:\Users\akshi\AppData\Local\Temp\ipykernel_22688\1526353581.py:24: DeprecationWarning: Firefox will soon stop logging to geckodriver.log by default; Specify desired logs with log_output
  driver = webdriver.Firefox(service=Service(GeckoDriverManager().install()), options = firefox_options)


InvalidArgumentException: Message: Malformed URL: URL constructor: /browse/info.do?cid=1086537 is not a valid URL.
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:189:5
InvalidArgumentError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:380:5
GeckoDriver.prototype.navigateTo@chrome://remote/content/marionette/driver.sys.mjs:849:11


In [12]:
### writing a csv 
import csv
filename = "gap_hypo2.csv"
rows = zip(product_names, Price,Count,Stars)
with open(filename, mode='w', newline = '',encoding='utf-8') as file:
    writer = csv.writer(file)
    header = ["names", "Price", "Count", "stars"]
    writer.writerow(header)
    writer.writerows(rows)
print(f"csv file '{filename}' has been created.")

csv file 'gap_hypo2.csv' has been created.


'C:\\Users\\akshi\\OneDrive\\Desktop\\Web Data Analytics\\Team Project'

In [11]:
product_names

['100% Organic Cotton T-Shirt',
 '100% Organic Cotton T-Shirt',
 'Waffle Crewneck T-Shirt',
 'Henley T-Shirt',
 'Henley T-Shirt',
 'Henley T-Shirt',
 '100% Organic Cotton T-Shirt',
 'Waffle Crewneck T-Shirt',
 'Waffle T-Shirt',
 '100% Organic Cotton T-Shirt',
 '100% Organic Cotton T-Shirt',
 'Waffle Crewneck T-Shirt',
 '100% Organic Cotton T-Shirt',
 'Waffle Crewneck T-Shirt',
 'Waffle Crewneck T-Shirt',
 '100% Organic Cotton T-Shirt',
 'Gap Arch Logo T-Shirt',
 'Gap Arch Logo T-Shirt',
 'Gap Arch Logo T-Shirt',
 'Gap Arch Logo T-Shirt',
 '100% Organic Cotton T-Shirt',
 'N/A',
 '100% Organic Cotton T-Shirt',
 'Waffle Crewneck T-Shirt',
 'Waffle T-Shirt',
 '100% Organic Cotton T-Shirt',
 '100% Organic Cotton T-Shirt',
 'Henley T-Shirt',
 '100% Organic Cotton T-Shirt',
 'Waffle T-Shirt',
 'Waffle Hoodie',
 'Waffle Hoodie',
 'Waffle Hoodie',
 'Waffle Henley T-Shirt',
 'Waffle Henley T-Shirt',
 'Waffle Henley T-Shirt',
 'Heavyweight Pocket T-Shirt',
 'Heavyweight Pocket T-Shirt',
 'Heavywe